#   Лаборатоная работа 2
***
***Выполнила:*** *Нечаева Анна Анатольвна, R3338*
***
**Цель работы:** построить модели линейной классификации и регрессии.

**Инструменты:** модели классификации (SGDClassifier) и регрессии (SGDRegressor) из пакета scikit-learn.


**Содержание:**

Для предложенных датасетов (один для классификации, другой для регрессии) построить соответствующие линейные модели. Для этого:

1. Воспользоваться результатами предыдущей лабораторной работы по препроцессингу данных.
2. Разбить данные на обучающую и тестовую выборки (например, в соотношении 80/20).
3. Построить классификатор и регрессор с различными вариантами архитектур.
4. Обучить модели на обучающих частях выборки.
5. Провести сравнительную оценку качества моделей на основе тестовой выборки


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

***
**1. Воспользуемся результатами предыдущей лабораторной работы по препроцессингу данных.**
***

In [12]:
# данные для регрессии
def read_data(path, filename):
    return pd.read_csv(os.path.join(path, filename))

adv_data = read_data('drive/MyDrive/ML_3_year_data', 'Advertising_new.csv')
adv_data = adv_data.drop('Unnamed: 0', axis=1)
adv_data.head()

,TV,radio,sales
0,1.692525,-3.197460,22.1
1,-0.474703,-3.096175,10.4
2,-0.793482,-2.650519,9.3
3,0.774723,-2.961128,18.5
4,1.116855,-5.020596,12.9


In [13]:
#  данные для классификации
def read_data(path, filename):
    return pd.read_csv(os.path.join(path, filename))

h_data = read_data('drive/MyDrive/ML_3_year_data', 'heart_new.csv')
h_data = h_data.drop('Unnamed: 0', axis=1)
h_data.head()

,age,sex,cp,thalach,exang,ca,thal,oldpeak_slope_1,target
0,2.642611,-82.697009,-35.212951,4.869293,-83.545412,-37.968538,-62.237454,-31.395502,1
1,-0.204919,-82.697009,-36.194604,6.490508,-83.545412,-37.968538,-60.594050,-30.445388,1
2,0.233162,-84.880656,-37.176258,5.833259,-83.545412,-37.968538,-60.594050,-32.691946,1
3,1.875968,-82.697009,-37.176258,6.096158,-83.545412,-37.968538,-60.594050,-33.167003,1
4,1.985488,-84.880656,-38.157911,5.438909,-81.396116,-37.968538,-60.594050,-33.325355,1


***
**2. Разбиение данных на обучающую и тестовую выборки.**
***

In [14]:
from sklearn.model_selection import train_test_split


# Для классификации
X_class = h_data.drop('target', axis=1)  # Признаки
y_class = h_data['target']  # Целевая переменная

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Для регрессии
X_reg = adv_data.drop('sales', axis=1)  # Признаки
y_reg = adv_data['sales']  # Целевая переменная

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
X_class

,age,sex,cp,thalach,exang,ca,thal,oldpeak_slope_1
0,2.642611,-82.697009,-35.212951,4.869293,-83.545412,-37.968538,-62.237454,-31.395502
1,-0.204919,-82.697009,-36.194604,6.490508,-83.545412,-37.968538,-60.594050,-30.445388
2,0.233162,-84.880656,-37.176258,5.833259,-83.545412,-37.968538,-60.594050,-32.691946
3,1.875968,-82.697009,-37.176258,6.096158,-83.545412,-37.968538,-60.594050,-33.167003
4,1.985488,-84.880656,-38.157911,5.438909,-81.396116,-37.968538,-60.594050,-33.325355
...,...,...,...,...,...,...,...,...
298,1.711687,-82.697009,-37.176258,4.978835,-83.545412,-37.968538,-60.594050,-33.139802
299,1.711687,-82.697009,-37.176258,4.978835,-83.545412,-37.968538,-60.594050,-33.139802
300,1.711687,-82.697009,-37.176258,4.978835,-83.545412,-37.968538,-60.594050,-33.139802
301,1.711687,-82.697009,-37.176258,4.978835,-83.545412,-37.968538,-60.594050,-33.139802


***
**3. Построение классификатора и регрессора с различными вариантами архитектур.**
***

Построим классификаторы с различными вариантами архитектур, включая разные функции потерь и виды регуляризации и определим лучшую модель, используя инструмент `'GridSearchCV'` из библиотеки `'sklearn.model_selection'`

In [15]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

# Задаем параметры для GridSearchCV для классификатора
param_grid_class = {
    'loss': ['perceptron', 'hinge', 'squared_hinge'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [1000, 2000],
}

seed = 7

np.random.seed(seed)

grid_search_class = GridSearchCV(SGDClassifier(random_state=seed), param_grid_class, cv=5, error_score='raise')
grid_search_class.fit(X_train_class, y_train_class)

best_classifier = grid_search_class.best_estimator_
best_p = grid_search_class.best_params_
best_p

{'alpha': 0.01, 'loss': 'perceptron', 'max_iter': 1000, 'penalty': 'l1'}

Аналогично, создаем регрессоры с использованием различных функций потерь и регуляризаций.

In [16]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV

# Задаем параметры для GridSearchCV для регрессора
param_grid_reg = {
    'loss': ['squared_error', 'huber', 'epsilon_insensitive'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [1000, 2000],
}

grid_search_reg = GridSearchCV(SGDRegressor(random_state=seed), param_grid_reg, cv=5)
grid_search_reg.fit(X_train_reg, y_train_reg)

best_regressor = grid_search_reg.best_estimator_
best_p = grid_search_reg.best_params_
best_p

{'alpha': 0.01, 'loss': 'squared_error', 'max_iter': 1000, 'penalty': 'l2'}

***
**4. Обучение модели и проведение сравнительной оценки**
***

In [17]:
from sklearn.metrics import accuracy_score, classification_report

best_classifier.fit(X_train_class, y_train_class)
y_pred_class = best_classifier.predict(X_test_class)

accuracy = accuracy_score(y_test_class, y_pred_class)
print(f'Accuracy классификатора: {accuracy}')
print(classification_report(y_test_class, y_pred_class))

Accuracy классификатора: 0.7704918032786885
              precision    recall  f1-score   support

           0       0.88      0.56      0.68        27
           1       0.73      0.94      0.82        34

    accuracy                           0.77        61
   macro avg       0.80      0.75      0.75        61
weighted avg       0.80      0.77      0.76        61



In [18]:
from sklearn.metrics import mean_squared_error, r2_score

best_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg = best_regressor.predict(X_test_reg)

mse = mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f'MSE регрессора: {mse}')
print(f'R² регрессора: {r2}')

MSE регрессора: 3.4138297989612014
R² регрессора: 0.8918428038961438
